#Crawling PTA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Nama : Akhmad Amanulloh

NIM  : 200411100099

Kelas : PPW A

In [ ]:
!pip install requests
!pip install html5lib
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=a617f562b880c1c23d4d7d42fa992d1fe6f4a4578930fdfb08a338bc6eeebaf5
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# mengirim permintaan HTTP dengan header user-agent yang ditentukan
def kirim_permintaan(header, url):
    return requests.get(url=url, headers={'User-Agent': header})

# menguraikan situs web menggunakan BeautifulSoup
def urai_situs(permintaan):
    return BeautifulSoup(permintaan.content, 'html5lib')

# mendapatkan tabel konten dari elemen web
def dapatkan_tabel_konten(elemen, tag, atribut):
    return elemen.find(tag, attrs=atribut)

# mengekstrak data dari sebuah halaman
def ekstrak_data(url_link, halaman):
    url_halaman = f"{url_link}/{halaman}"
    r = kirim_permintaan("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", url_halaman)
    tabel = dapatkan_tabel_konten(urai_situs(r), "div", {"id": "wrapper"})
    artikel = []

    for artikel_row in tabel.findAll("a", attrs={"class": "gray button"}):
        r_artikel = kirim_permintaan("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36", artikel_row["href"])
        tabel_artikel = dapatkan_tabel_konten(urai_situs(r_artikel), "li", {"data-id": "id-1", "data-cat": "#luxury"})
        judul = tabel_artikel.find('a', attrs={"class": "title", "href": "#"}).string
        penulis, pembimbing1, pembimbing2, abstrak = "", "", "", tabel_artikel.find('p', attrs={"align": "justify"}).string

        for span in tabel_artikel.find_all('span'):
            span_teks = span.get_text(strip=True)
            if "Penulis :" in span_teks:
                penulis = span_teks.replace("Penulis :", "").strip()
            elif "Dosen Pembimbing I :" in span_teks:
                pembimbing1 = span_teks.replace("Dosen Pembimbing I :", "").strip()
            elif "Dosen Pembimbing II :" in span_teks:
                pembimbing2 = span_teks.replace("Dosen Pembimbing II :", "").strip()

        artikel.append([judul, penulis, pembimbing1, pembimbing2, abstrak])

    return artikel

# URL situs web sumber data
url_link = "https://pta.trunojoyo.ac.id/c_search/byprod/10"

# Mendapatkan tabel konten utama dari halaman web
tabel = dapatkan_tabel_konten(urai_situs(kirim_permintaan("Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1", url_link)), "div", {"id": "wrapper"})

# Menemukan dan menghitung jumlah halaman menggunakan pagination
pagination = tabel.findAll("a", attrs={"class": "pag_button"})
total_halaman = int(pagination[-1]["href"].split("/")[-1])

# Inisialisasi list menyimpan data
artikel = []

# Loop melalui setiap halaman untuk mengambil data
for halaman in range(1, total_halaman + 1):
    artikel += ekstrak_data(url_link, halaman)

# Membuat DataFrame dari data yang diambil menyimpannya ke file CSV
df = pd.DataFrame(artikel, columns=["Judul", "Penulis", "Pembimbing I", "Pembimbing II", "Abstrak"])
df.to_csv('PTA.csv', index=False)


In [ ]:
pd.read_csv('/content/PTA.csv')

,Judul,Penulis,Pembimbing I,Pembimbing II,Abstrak
0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...
1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...
4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...,...
853,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Rachmad Agung Pambudi,"Eka Mala Sari Rochman, S.Kom., M.Kom","Sri Herawati, S.Kom., M.Kom",Investasi saham selama ini memiliki resiko ker...
854,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Nadila Hidayanti,"Achmad Jauhari, S.T., M.Kom","Ika Oktavia Suzanti, S.Kom., M.Cs",Information Retrieval (IR) merupakan pengambil...
855,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Afni Sakinah,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Moch. Kautsar Sophan, S.Kom., M.MT.",Klasifikasi citra merupakan proses pengelompok...
856,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Friska Fatmawatiningrum,"Dr. Indah Agustien Siradjuddin, S.Kom., M.Kom.","Prof. Dr. Arief Muntasa, S.Si., M.MT.",Identifikasi atribut pejalan kaki merupakan sa...
